In [1]:
from Parameters import *

Welcome to JupyROOT 6.24/06


/uscms_data/d3/ahayrape/mamba/envs/anal/lib/python3.9/site-packages/numba/core/cpu.py:97: UserWarning: Numba extension module 'awkward.numba' failed to load due to 'FileNotFoundError([Errno 2] No such file or directory: '/uscms_data/d3/ahayrape/mamba/envs/anal/lib/python3.9/site-packages/numpy-1.23.2.dist-info/METADATA')'.
  numba.core.entrypoints.init_all()


In [2]:
MuonSystem = "DT" # "CSC"
denominator = "mb1_adjacent" # "cut_based_ID"

# file = open(f"Output_{MuonSystem}.pickle", "rb")
file = open(f'./Pickles/Output_TRACKS_50_{MuonSystem}.pickle', "rb")

data = pickle.load(file)

file.close()

In [4]:
import pandas as pd
denominator = "muonSF with eventHLT_cut" # "muonSF with posttrigger"
datasets = [i for i in data.keys() if "_cuts" not in i if "_vars" not in i]
for d in range(len(datasets)):
    output_folder = f"Cutflow_tables_{MuonSystem}"
    os.system(f"mkdir -p {output_folder}")
    dataset  = datasets[d]
    values   = list(data[f"{dataset}_vars"].values())
    keys     = [k for k in list(data[f"{dataset}_vars"].keys()) if k.split(" with ")[0] == "muonSF"]
    cf_keys  = [k.split(" with ")[1] for k in keys]
    values   = [data[f"{dataset}_vars"][k] for k in keys]
    n_events = [sum(v) for v in values]
    eff      = []
    cum_eff  = []
    for j in range(len(keys)):
        if j == 0:
            eff.append("")
        else:
            eff.append(100 * n_events[j] / n_events[j-1])
        ind = keys.index(denominator)
        c = 100 * n_events[j] / n_events[ind]
        if j < ind:
            c = ""
        cum_eff.append(c)

    output_dataset = {}
    output_dataset = {"Cuts" : cf_keys, "Number of events": n_events, "Cumulative efficiency": cum_eff, "Efficiency": eff}
    df = pd.DataFrame(output_dataset)
    df.to_csv(f"./{output_folder}/{dataset}.csv")